In [1]:
from EDA_preprocessing import preprocess_data

corrected = "./data/corrected"
kddcup_data =  "./data/kddcup.data.corrected"

kddcup_data_10_percent_corrected = "./data/kddcup.data_10_percent_corrected"
kddcup_newtestdata_10_percent_unlabeled = "./data/kddcup.newtestdata_10_percent_unlabeled"

kddcup_testdata_unlabeled = "./data/kddcup.testdata.unlabeled"
kddcup_testdata_unlabeled_10_percent = "./data/kddcup.testdata.unlabeled_10_percent"

correspondence = "./data/training_attack_types.txt"

In [ ]:
#%%notify
X_train, y_train, X_test, y_test = preprocess_data(kddcup_data,
                                                   kddcup_testdata_unlabeled_10_percent,
                                                   "./data/headers",
                                                   "./data/training_attack_types.txt")

In [ ]:
X_train.to_csv('X_train.csv')
X_test.to_csv("X_test.csv")
y_train.to_csv("y_train.csv")
y_test.to_csv('y_test.csv')

In [1]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, accuracy_score
import numpy as np
from scorer import classification_report
import pandas as pd


In [2]:

X_test = pd.read_csv('X_train.csv', header=0, index_col=0)
X_train = pd.read_csv("X_test.csv", header=0, index_col=0)
y_train = pd.Series.from_csv("y_test.csv", header=None, index_col=0)
y_test = pd.Series.from_csv('y_train.csv', header=None, index_col=0)

target_names = ['normal', 'probing', 'dos','u2r' ,'r2l']

In [3]:
print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(247247, 37)
(247247,)
(827736, 37)
(827736,)


In [4]:
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import f1_score, accuracy_score
import numpy as np

In [5]:
clf = GaussianNB()
clf.fit(X_train, y_train)

predict = clf.predict(X_test)

print (classification_report(y_test, predict, digits=4, target_names=target_names))

             precision    recall  f1-score   support

     normal     0.9990    0.5197    0.6837    625708
    probing     0.0891    0.9857    0.1633     10619
        dos     0.9919    0.9852    0.9886    190582
        u2r     0.0001    0.6429    0.0003        42
        r2l     0.0289    0.4917    0.0546       785

avg / total     0.9847    0.6328    0.3781    827736



In [6]:
from sklearn.svm import SVC
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import make_scorer

In [7]:
clf = SVC(kernel='linear')
clf.fit(X_train, y_train)

predict = clf.predict(X_test)

print (classification_report(y_test, predict, digits=4, target_names=target_names))

             precision    recall  f1-score   support

     normal     0.9930    0.9989    0.9960    625708
    probing     0.9671    0.7633    0.8532     10619
        dos     0.9975    0.9939    0.9957    190582
        u2r     0.0000    0.0000    0.0000        42
        r2l     0.0000    0.0000    0.0000       785

avg / total     0.9927    0.9938    0.5690    827736



/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/usr/local/lib/python2.7/dist-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [8]:
#%%notify

params = {
    'kernel' : ['rbf', 'poly', 'sigmoid', 'linear'],
    'C' : [0.1, 1, 5, 10, 15],
}
clf = GridSearchCV(SVC(), params, n_jobs=2, verbose=2)
clf.fit(X_train[:10000], y_train[:10000])

print (clf.best_estimator_)
clf = clf.best_estimator_

predict = clf.predict(X_test)

Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] kernel=rbf, C=0.1 ...............................................
[CV] kernel=rbf, C=0.1 ...............................................
[CV] ................................ kernel=rbf, C=0.1, total=   1.3s
[CV] kernel=rbf, C=0.1 ...............................................
[CV] ................................ kernel=rbf, C=0.1, total=   1.4s
[CV] kernel=poly, C=0.1 ..............................................
[CV] ................................ kernel=rbf, C=0.1, total=   0.8s
[CV] kernel=poly, C=0.1 ..............................................
[CV] ............................... kernel=poly, C=0.1, total=   2.2s
[CV] kernel=poly, C=0.1 ..............................................
[CV] ............................... kernel=poly, C=0.1, total=   2.1s
[CV] kernel=sigmoid, C=0.1 ...........................................
[CV] ............................ kernel=sigmoid, C=0.1, total=   1.1s
[CV] kernel=sigm

[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:   23.2s


[CV] ................................. kernel=rbf, C=10, total=   0.2s
[CV] kernel=poly, C=10 ...............................................
[CV] ................................ kernel=poly, C=10, total=   0.5s
[CV] kernel=poly, C=10 ...............................................
[CV] ................................ kernel=poly, C=10, total=   0.5s
[CV] kernel=sigmoid, C=10 ............................................
[CV] ............................. kernel=sigmoid, C=10, total=   0.3s
[CV] kernel=sigmoid, C=10 ............................................
[CV] ................................ kernel=poly, C=10, total=   0.6s
[CV] kernel=sigmoid, C=10 ............................................
[CV] ............................. kernel=sigmoid, C=10, total=   0.3s
[CV] kernel=linear, C=10 .............................................
[CV] ............................. kernel=sigmoid, C=10, total=   0.3s
[CV] kernel=linear, C=10 .............................................
[CV] .

[Parallel(n_jobs=2)]: Done  60 out of  60 | elapsed:   28.4s finished


SVC(C=15, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)


In [9]:
print (classification_report(y_test, predict, digits=4, target_names=target_names))

             precision    recall  f1-score   support

     normal     0.9933    0.9993    0.9963    625708
    probing     0.9842    0.8048    0.8855     10619
        dos     0.9973    0.9921    0.9947    190582
        u2r     0.0000    0.0000    0.0000        42
        r2l     0.0000    0.0000    0.0000       785

avg / total     0.9931    0.9941    0.5753    827736



In [10]:
from sklearn.ensemble import RandomForestClassifier

In [11]:
params = {
    'n_estimators' : [10,25,50,100],
    'max_depth' : [10,25,50,100,None],
}

clf = GridSearchCV(RandomForestClassifier(), params, n_jobs=2, verbose=2)
clf.fit(X_train, y_train)

print (clf.best_estimator_)
clf = clf.best_estimator_


predict = clf.predict(X_test)



Fitting 3 folds for each of 20 candidates, totalling 60 fits
[CV] n_estimators=10, max_depth=10 ...................................
[CV] n_estimators=10, max_depth=10 ...................................
[CV] .................... n_estimators=10, max_depth=10, total=   1.9s
[CV] n_estimators=10, max_depth=10 ...................................
[CV] .................... n_estimators=10, max_depth=10, total=   2.0s
[CV] n_estimators=25, max_depth=10 ...................................
[CV] .................... n_estimators=10, max_depth=10, total=   2.9s
[CV] n_estimators=25, max_depth=10 ...................................
[CV] .................... n_estimators=25, max_depth=10, total=   5.2s
[CV] n_estimators=25, max_depth=10 ...................................
[CV] .................... n_estimators=25, max_depth=10, total=   4.9s
[CV] n_estimators=50, max_depth=10 ...................................
[CV] .................... n_estimators=25, max_depth=10, total=   6.2s
[CV] n_estimator

[Parallel(n_jobs=2)]: Done  37 tasks      | elapsed:  3.7min


[CV] ................... n_estimators=10, max_depth=100, total=   2.4s
[CV] n_estimators=25, max_depth=100 ..................................
[CV] ................... n_estimators=25, max_depth=100, total=   5.7s
[CV] n_estimators=25, max_depth=100 ..................................
[CV] ................... n_estimators=100, max_depth=50, total=  22.5s
[CV] n_estimators=25, max_depth=100 ..................................
[CV] ................... n_estimators=25, max_depth=100, total=   6.8s
[CV] n_estimators=50, max_depth=100 ..................................
[CV] ................... n_estimators=25, max_depth=100, total=   5.4s
[CV] n_estimators=50, max_depth=100 ..................................
[CV] ................... n_estimators=50, max_depth=100, total=  12.3s
[CV] n_estimators=50, max_depth=100 ..................................
[CV] ................... n_estimators=50, max_depth=100, total=  12.9s
[CV] n_estimators=100, max_depth=100 .................................
[CV] .

[Parallel(n_jobs=2)]: Done  60 out of  60 | elapsed:  6.5min finished


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=50, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=50, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)
             precision    recall  f1-score   support

     normal     0.9982    0.9998    0.9990    625708
    probing     0.9937    0.9877    0.9906     10619
        dos     1.0000    0.9967    0.9983    190582
        u2r     1.0000    0.1190    0.2128        42
        r2l     0.9343    0.5796    0.7154       785

avg / total     0.9985    0.9985    0.7832    827736



In [ ]:
print (classification_report(y_test, predict, digits=4, target_names=target_names))

In [12]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
%%notify

params = {
    'n_estimators' : [50,100,200,300],
    'learning_rate' : [1,2,0.5,.1],
}

clf = GridSearchCV(AdaBoostClassifier(), params, n_jobs=2, verbose=2)
clf.fit(X_train, y_train)

print (clf.best_estimator_)
clf = clf.best_estimator_

predict = clf.predict(X_test)


In [ ]:
print (classification_report(y_test, predict, digits=4, target_names=target_names))

In [15]:
from sklearn.neighbors import KNeighborsClassifier

In [16]:
%%notify

clf = KNeighborsClassifier(n_neighbors=len(np.unique(y_train)))
clf.fit(X_train[:20000], y_train[:20000])


predict = clf.predict(X_test)


<IPython.core.display.Javascript object>

In [17]:
print (classification_report(y_test, predict, digits=4, target_names=target_names))

             precision    recall  f1-score   support

     normal     0.9965    0.9992    0.9979    625708
    probing     0.9752    0.9216    0.9476     10619
        dos     0.9983    0.9944    0.9963    190582
        u2r     0.0000    0.0000    0.0000        42
        r2l     0.7694    0.4675    0.5816       785

avg / total     0.9964    0.9965    0.7047    827736

